In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.widgets import Slider
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import warnings
import base64
import io
warnings.filterwarnings('ignore')
%matplotlib inline


<h1 Align='center'>Analysis Report</h1>

In [ ]:
# Allows to display all the columns in the dataframe
pd.set_option('display.max_columns', None)

In [ ]:
# Filepath
file_path = 'FILE_PATH_PLACEHOLDER'

In [ ]:
# Read csv and convert it to Pandas dataframe
data = pd.read_csv(file_path)

In [ ]:
def export_report(df):
    # Custom CSS for table layout and responsiveness
    css = """
    <style>
    /* General table styling */
    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: auto;
        max-width: 100%;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border: 1px solid #ddd;
        word-wrap: break-word;
    }
    
    /* Responsive table container for small screens */
    .scrollable-table {
        display: block;
        width: 100%;
        overflow-x: auto;
        max-width: 100%;
        white-space: nowrap;
    }

    /* Responsive adjustments for small screens */
    @media only screen and (max-width: 768px) {
        th, td {
            min-width: 120px;
            font-size: 12px;
        }
    }
    
    /* Print-specific styles */
    @media print {
        .navbar, .footer, .sidebar, .button { 
            display: none; 
        }

        .scrollable-table {
            overflow: visible;
        }
        
        body {
            margin: 0;
            padding: 0;
        }
        
        table {
            page-break-inside: auto;
            width: 100%;
        }
        
        tr {
            page-break-inside: avoid;
            page-break-after: auto;
        }
    }
    </style>
    """
    
    
    # Otherwise, convert the regular DataFrame to HTML
    df_html = f'<div class="scrollable-table">{df.to_html()}</div>'
    
    # Combine CSS and DataFrame HTML
    full_html = css + df_html
    display(HTML(full_html))

In [ ]:
def export_report_styled(df_or_styler):
    css = """
    <style>
    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: auto;
        max-width: 100%;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border: 1px solid #ddd;
        word-wrap: break-word;
    }
    .scrollable-table {
        display: block;
        width: 100%;
        overflow-x: auto;
        max-width: 100%;
        white-space: nowrap;
    }
    @media only screen and (max-width: 768px) {
        th, td {
            min-width: 120px;
            font-size: 12px;
        }
    }
    @media print {
        .scrollable-table {
            overflow: visible;
        }
        body {
            margin: 0;
            padding: 0;
        }
        table {
            page-break-inside: auto;
            width: 100%;
        }
        tr {
            page-break-inside: avoid;
            page-break-after: auto;
        }
    }
    </style>
    """

    # Check if the input is a Styler object
    if isinstance(df_or_styler, pd.io.formats.style.Styler):
        # Use to_html() on Styler to retain styles
        df_html = f'<div class="scrollable-table">{df_or_styler.to_html()}</div>'
    else:
        # Otherwise, convert the regular DataFrame to HTML
        df_html = f'<div class="scrollable-table">{df_or_styler.to_html()}</div>'
    
    # Combine CSS and DataFrame HTML
    full_html = css + df_html
    display(HTML(full_html))

In [ ]:
for col in data.columns:
  # Convert Excel division erros to numpy NaN
   data[col] = data[col].replace('#DIV/0!', np.nan)

In [ ]:
#data.iloc[:, 15:] = data.iloc[:, 15:].astype(float)
for column in data.iloc[:, 15:]:
    # Check if the column contains non-numeric values
    if data[column].dtype == 'object':
        # Convert non-numeric values to float
        data[column] = data[column].astype(np.float64)


In [ ]:
style_data = list(data.columns[15:].values.tolist())

In [ ]:
#((P2-Q2)/(0.5*(P2+Q2)))*100
# Calculate AMPRD, MEAN and Precision of Ag
for i in range(15, len(data.columns)):
  if 'Duplicate' not in data.columns[i]:
    data['AMPRD of ' + data.columns[i]] = (np.abs(data[data.columns[i]] - data['Duplicate ' + data.columns[i]])/(0.5*(data[data.columns[i]] + data['Duplicate ' + data.columns[i]]))*100)
    data['Mean of ' + data.columns[i]] = data[[data.columns[i], 'Duplicate ' + data.columns[i]]].mean(axis=1)
    data['Precision of ' + data.columns[i]] = ((data[data.columns[i]] - data['Duplicate ' + data.columns[i]]) / (0.5 * (data[data.columns[i]] + data['Duplicate ' + data.columns[i]]))) * 100


In [ ]:
# Map each AMPRD of every element and its duplicate to corresponding element and duplicate
column_mapping = {}
for i in style_data:
  # Map duplicate of element to its AMPRD
  if 'Duplicate' in i:
    column_mapping[i] = 'AMPRD of ' + i.split(' ')[1] # Split 'Duplicate element' so that we can get only the element and get 'AMPRD of element'
  # Map element to its AMPRD
  else:
    column_mapping[i] = 'AMPRD of ' + i
#print(column_mapping)

In [ ]:
def style_based_on_other_column(row):
    """
      This function styles a column based on the condition of the AMPRD of the current element on the same row being greater than 10

      args:
        row: a row of the dataframe
      returns:
        a list of styles for each cell in the row
    """
    # Generate the styles for each column in the row based on the mapping
    styles = []
    for col in row.index:
        # Check if the current column has a corresponding reference column
        reference_column = column_mapping.get(col, None)
        if reference_column and pd.notna(row[reference_column]) and row[reference_column] > 10:
            styles.append('background-color: red')  # Apply style based on the reference value
        else:
            styles.append('')  # No style applied
    return styles

# Apply the conditional styling to the DataFrame

df_rounded = data.copy()
numeric_columns = data.select_dtypes(include='number').columns
df_rounded[numeric_columns] = data[numeric_columns].round(2)
styled_df = df_rounded.style.apply(style_based_on_other_column, axis=1).set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [('font-size', '8pt')]}]).format({col: "{:.2f}" for col in numeric_columns}).hide(axis='index')


# Display the styled DataFrame
#export_report_styled(styled_df)

In [ ]:
hidden_columns = data.columns[data.columns.get_loc('AMPRD of ' + data.columns[15]):].tolist()

In [ ]:
styled_df = styled_df.hide(axis='columns', subset=hidden_columns)
export_report_styled(styled_df)

<h1 align='center'>Statistics</h1>

In [ ]:
def statistics(elements):
  """
    This function returns a dictionary with the statistics of each element

    args:
      elements: a list of elements to be analyzed
    returns:
      a dictionary with the statistics of each element

  """
  # Create an empty dictionary to store the statistics
  min_dict = {}
  # Loop through each element in the list
  for element in data[elements]:
    # Calculate the statistics for the current element
    count = data[element].count()
    std = data[element].std()
    max = data[element].max()
    min = data[element].min()
    mean = data[element].mean()
    # Check if the element is a duplicate
    if 'Duplicate' in element:
      # Calculate the precision, bias, correlation, cross correlation, and percent less than 10, 15, and 20
      precision = (np.sqrt(((data[element.split(' ')[1]] - data[element]) ** 2).sum() / (2 * count)) / data[[element.split(' ')[1], element]].stack().mean()) * 100
      bias = ((np.sum(data[element.split(' ')[1]]) - np.sum(data[element])) / np.sum(data[element])) * 100
      correlation = data[[element, element.split(' ')[1]]].corr(method='pearson').iloc[0, 1]
      cross_correlation = (np.nansum(data[element.split(' ')[1]] * data[element])) / np.sqrt(np.nansum(data[element.split(' ')[1]]**2) * np.nansum(data[element]**2))
      percent_less_10 = np.sum(data['AMPRD of '+ element.split(' ')[1]] <= 10) / count * 100
      percent_less_15 = np.sum(data['AMPRD of '+ element.split(' ')[1]] <= 15) / count * 100
      percent_less_20 = np.sum(data['AMPRD of '+ element.split(' ')[1]] <= 20) / count * 100
    # If the element is not a duplicate: precision, bias, correlation, cross correlation, and percent less than 10, 15, and 20 will not be calculated
    else:
      precision = None
      bias = None
      correlation = None
      cross_correlation = None
      percent_less_10 = None
      percent_less_15 = None
      percent_less_20 = None
    # Store the statistics in the dictionary
    min_dict[element] = [count, std, max, min, mean, precision, bias, correlation, cross_correlation, percent_less_10, percent_less_15, percent_less_20]
  return min_dict



In [ ]:
end_elems = data.columns.get_loc('AMPRD of Ag')

In [ ]:
stat_data = list(data.columns[15:end_elems].values.tolist()) # List of the elements that will be used to get the statistics
# Get the statistics of the elements
stats = statistics(stat_data)
# Create a dataframe with the statistics of the elements
stats_df = pd.DataFrame.from_dict(stats, orient='index', columns=['Count', 'Std Dev', 'Max', 'Min', 'Mean', 'Precision', 'Bias',  'Correlation', 'Cross Correlation', '% < 10%', '% < 15%', '% < 20%'])
stats_df_rounded = stats_df.copy()
numeric_columns = stats_df.select_dtypes(include='number').columns
stats_df_rounded[numeric_columns].round(2)
styled_stats = stats_df_rounded.style.set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [{'font-size', '8pt'}]}]).format({col: "{:.2f}" for col in numeric_columns})
export_report(styled_stats)

In [ ]:
# Function to create a downloadable Excel file with two sheets
def create_download_link_multi_sheet(df1, df2, filename="Report.xlsx"):
    # Save the DataFrames to an in-memory Excel file with two sheets
    excel_buffer = io.BytesIO()
    with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
        df1.to_excel(writer, sheet_name='Sheet1', index=False)
        df2.to_excel(writer, sheet_name='Sheet2', index=False)
    
    excel_buffer.seek(0)  # Go to the beginning of the in-memory file
    
    # Convert the Excel file to a base64-encoded string
    b64 = base64.b64encode(excel_buffer.read()).decode()
    
    # Create the HTML button with the download link
    return HTML(f"""
        <a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">
            <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;">
                Download Excel
            </button>
        </a>
    """)

# Display the button
create_download_link_multi_sheet(styled_df, stats_df)

In [ ]:
data['Certficate Date'] = pd.to_datetime(data['Certficate Date']) # Convert the Certficate Date column to datetime

<h1 align='center'>Elements and Duplicates Graph</h1>

In [ ]:
def plot_elements_and_duplicates(elements):
    fig = go.Figure()

    precision_range = 10

    for element in elements:
        #fig = go.Figure()
        fig.add_trace(go.Scatter(x=data[element], y=data['Duplicate ' + element], mode='markers', name='Duplicates', visible=False, marker=dict(color='red')))
        fig.add_trace(go.Scatter(x=[0, stats_df.loc[element]['Max']], y=[0, stats_df.loc[element]['Max']], mode='lines', name='1:1 Line', visible=False, line=dict(color='red')))
        fig.add_trace(go.Scatter(x=[0, stats_df.loc[element]['Max']], y=[0, (stats_df.loc[element]['Max'] * (1 + (precision_range / 100)))], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False, line=dict(dash='dash', color='blue')))
        fig.add_trace(go.Scatter(x=[0, stats_df.loc[element]['Max']], y=[0, (stats_df.loc[element]['Max'] * (1 - (precision_range / 100)))], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False, line=dict(dash='dash', color='blue')))

    for i in range(4):
        fig.data[i].visible = True

    # Create sliders for each element
    sliders = []
    for element_index, element in enumerate(elements):
      slider = {
          'active': 2,
          'currentvalue': {"prefix": "Precision Envelope: ", "visible": True},
          'pad': {'t': 50},
          'steps': []
      }

      # Slider steps for precision envelope
      for i in range(0, 51, 5):
        slider_step = {
            'label': str(i),
            'method': 'restyle',
            'args': [{
                'y': [
                      data['Duplicate ' + element].tolist(),  # Update the y-values dynamically
                      [0, stats_df.loc[element]['Max']],  # 1:1 line based on element
                      [0, (stats_df.loc[element]['Max'] * (1 + (i / 100)))],  # Upper bound precision envelope
                      [0, (stats_df.loc[element]['Max'] * (1 - (i / 100)))]   # Lower bound precision envelope
                ],
                'name': [
                        fig.data[0].name,
                        fig.data[1].name,
                        f'±{i}% Precision Envelope',
                        f'±{i}% Precision Envelope'
                ]
            },
            [element_index * 4, element_index * 4 + 1, element_index * 4 + 2, element_index * 4 + 3] # Update only this element
          ]
        }
        slider['steps'].append(slider_step)

      sliders.append(slider)



    dropdown_buttons = []

    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)},
                     {'title': f"El Pilar Project {data['QCType'][0]} @ ALS",
                      'xaxis.title.text': f"{element}",
                      'yaxis.title.text': f"Duplicate {element}",
                      'sliders': [sliders[i]]

                      }
                     ],
            'label': element,
            'method': 'update'
        })

        dropdown_buttons[i]['args'][0]['visible'][i * 4:i * 4 + 4] = [True, True, True, True]

    fig.update_layout(
        updatemenus = [{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5},
        }],
        title=f"El Pilar Project {data['QCType'][0]} @ ALS",
        xaxis_title=elements[0],
        yaxis_title=f'Duplicate {elements[0]}',
        width=900,
        height=600,

    )

    fig.update_layout(sliders=[sliders[0]])

    fig.show()





        

In [ ]:
plot_data = [i for i in stat_data if 'Duplicate' not in i] # List of the elements that will be plotted
plot_elements_and_duplicates(plot_data) # Plot the elements and their duplicates


<h1 align='center'>Mean and Precision Graph</h1>

In [ ]:
def plot_mean_precision(elements):
    fig = go.Figure()

    # Initial precision envelope range (10%)
    precision_range = 10

    for element in elements:
        #fig = go.Figure()
        fig.add_trace(go.Scatter(x=data['Mean of ' + element], y=data['Precision of ' + element], mode='markers', name='Duplicates', visible=False, marker=dict(color='red')))
        fig.add_trace(go.Scatter(x=[(data[[element, 'Duplicate '+ element]].max()).max(), 0], y=[0, 0], mode='lines', name='1:1 Line', visible=False, line=dict(color='red')))
        fig.add_trace(go.Scatter(x=[0, (data[[element, 'Duplicate ' + element]].max()).max()], y=[precision_range, precision_range], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False, line=dict(dash='dash', color='blue')))
        fig.add_trace(go.Scatter(x=[0, (data[[element, 'Duplicate ' + element]].max()).max()], y=[-precision_range, -precision_range], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False,  line=dict(dash='dash', color='blue')))

    for i in range(4):
        fig.data[i].visible = True

    # Create sliders for each element 
    sliders = []
    for element_index, element in enumerate(elements): 
      slider = {
          'active': 2,
          'currentvalue': {"prefix": "Precision Envelope: ", "visible": True},
          'pad': {'t': 50},
          'steps': []
      }

      for i in range(0, 51, 5):
        slider_step = {
            'label': str(i), 
            'method': 'restyle', 
            'args': [{
                'y': [
                            data['Precision of ' + element].tolist(),
                            [0, 0],
                            [i, i],
                            [-i, -i],
                        ],
                'name': [
                            fig.data[0].name,
                            fig.data[1].name,
                            f'{i}% Precision Envelope',
                            f'{i}% Precision Envelope',
                        ]
            }, 
            [element_index * 4, element_index * 4 + 1, element_index * 4 + 2, element_index * 4 + 3]
          ]
        }
        slider['steps'].append(slider_step)

      sliders.append(slider)

    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)},
                     {'title': f"El Pilar Project {data['QCType'][0]} @ ALS",
                      'xaxis.title.text': f"Mean of {element}",
                      'yaxis.title.text': f"Precision of {element}",
                      'sliders': [sliders[i]]

                      }
                     ],
            'label': element,
            'method': 'update'
        })

        dropdown_buttons[i]['args'][0]['visible'][i * 4:i * 4 + 4] = [True, True, True, True]

    fig.update_layout(
        updatemenus = [{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5}
        }],
        title=f"El Pilar Project {data['QCType'][0]} @ ALS",
        xaxis_title=f"Mean of {elements[0]}",
        yaxis_title=f"Precision of {elements[0]}",
        width=900,
        height=600,
    )

    fig.update_layout(sliders=[sliders[0]])

    fig.show()


In [ ]:
plot_data = [i for i in stat_data if 'Duplicate' not in i] # List of the elements that will be plotted
plot_mean_precision(plot_data)  # Plot the mean of the elements and their duplicate



<h1 align='center'>Dates Graph</h1>

In [ ]:
def plot_dates(elements):
    fig = go.Figure()
    x_data = data['Certficate Date']
    precision_range = 10
    # Add traces for each element
    for element in elements:
        fig.add_trace(go.Scatter(x=x_data, y=data['Precision of ' + element], mode='markers', name=f'{element} Duplicates', visible=False, marker=dict(color='red')))
        fig.add_trace(go.Scatter(x=[x_data.max(), x_data.min()], y=[0, 0], mode='lines', name='1:1 Line', visible=False, line=dict(color='red')))
        fig.add_trace(go.Scatter(x=[x_data.max(), x_data.min()], y=[precision_range, precision_range], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False, line=dict(dash='dash', color='blue')))
        fig.add_trace(go.Scatter(x=[x_data.max(), x_data.min()], y=[-precision_range, -precision_range], mode='lines', name=f'±{precision_range}% Precision Envelope', visible=False, line=dict(dash='dash', color='blue')))

    # Make the first element visible by default
    for i in range(4):
        fig.data[i].visible = True

    # Create sliders for each element 
    sliders = []
    for element_index, element in enumerate(elements):
      slider = {
          'active': 2, 
          'currentvalue': {"prefix": "Precision Envelope: ", "visible": True},
          'pad': {'t': 50},
          'steps': []
      }

      for i in range(0, 51, 5):
        slider_step = {
            'label': str(i), 
            'method': 'restyle', 
            'args': [{
                 'y': [
                             data['Precision of ' + element].tolist(),
                             [0,0],
                             [i,i],
                             [-i,-i]
                         ],
                  'name': [
                             f'{element} duplicates',
                             fig.data[1].name,
                             f'{i}% Precision Envelope',
                             f'{i}% Precision Envelope',
                         ]
            },
            [element_index * 4, element_index * 4 + 1, element_index * 4 + 2, element_index * 4 + 3]
          ]
        }
        slider['steps'].append(slider_step)

      sliders.append(slider)


    # Create dropdown buttons for each element
    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)},  # Start with all traces hidden
                     {'title': f"El Pilar Project {data['QCType'][0]} @ ALS - {element}",
                      'xaxis.title.text': 'Certificate Date',
                      'yaxis.title.text': f"Precision of {element}",  # Update title and axis
                      'sliders': [sliders[i]]}],
            'label': element,
            'method': 'update'
        })

        # Set the relevant traces for each element visible (4 traces per element)
        dropdown_buttons[i]['args'][0]['visible'][i * 4:i * 4 + 4] = [True, True, True, True]  # Show the 4 relevant traces for this element

    # Add the dropdown menu to the layout
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5},
        }],
        title=f"El Pilar Project {data['QCType'][0]} @ ALS - {elements[0]}",  # Default to first element
        xaxis_title='Certificate Date',  # Fixed x-axis title
        yaxis_title=f"Precision of {elements[0]}",  # Default to the first element
        width=900,
        height=600,
        
    )

    fig.update_layout(sliders=[sliders[0]])

    fig.show()




In [ ]:
plot_data = [i for i in stat_data if 'Duplicate' not in i] # List of the elements that will be plotted
plot_dates(plot_data) # Plot the elements and their duplicates

<h1 align='center'>RDP Graphs</h1>

In [ ]:
# Function to plot multiple elements with a dropdown for RDP vs Grade
def plot_rdp_vs_grade(elements):
    fig = go.Figure()

    # Add a scatter plot for each element, but keep them hidden initially
    for element in elements:
        fig.add_trace(go.Scatter(x=data['Mean of ' + element], y=data['AMPRD of ' + element], mode='markers', name=element, visible=False, marker=dict(color='blue')))

    # Set the first element visible by default
    fig.data[0].visible = True

    # Create dropdown options for each element
    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)}, 
                     {'title': f"RDP Vs. Grade - {element}",
                      'xaxis.title': f"{element} ppm",
                      'yaxis.title': "RDP%"}],
                      
            'label': element,
            'method': 'update'
        })
        
        # Set only the corresponding scatter plot visible
        dropdown_buttons[i]['args'][0]['visible'][i] = True

    # Add the dropdown to the layout
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5}, 
        }],
        title=f"RDP Vs. Grade",
        xaxis_title=f"{elements[0]} ppm",  # Default to the first element
        yaxis_title="RDP%",  # Fixed y-axis title
        width=900, 
        height=600,
    )
    
    fig.show()

# Define elements to plot
plot_data = [i for i in stat_data if 'Duplicate' not in i]

# Call the function with the list of elements
plot_rdp_vs_grade(plot_data)



In [ ]:
def plot_rdp_vs_depth(elements): 
    fig = go.Figure()
    for element in elements: 
       #fig = go.Figure()
        fig.add_trace(go.Scatter(x=data['From'], y=data['AMPRD of ' + element ], mode='markers', visible=False, marker=dict(color='blue')))
    
    fig.data[0].visible = True

    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)}, 
                     {'title': f"RDP Vs. Depth - {element}",
                      'xaxis.title': f"Duplicates Sample Depth (m)",
                      'yaxis.title': "RDP%"}],
            'label': element, 
            'method': 'update', 
        })

        dropdown_buttons[i]['args'][0]['visible'][i] = True
    
     # Add the dropdown to the layout
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5}, 
        }],
        title=f"RDP Vs. Depth",
        xaxis_title=f"Duplicate Sample Depth (m)",  # Default to the first element
        yaxis_title="RDP%",  # Fixed y-axis title
        width=900, 
        height=600, 
    )
    
    fig.show()

plot_data = [i for i in stat_data if 'Duplicate' not in i]
plot_rdp_vs_depth(plot_data)

In [ ]:

# Function to plot multiple elements with a dropdown
def plot_rdp_vs_ddh(elements):
    fig = go.Figure()
    
    # Add a scatter plot for each element, but keep them hidden initially
    for element in elements:
        fig.add_trace(go.Scatter(x=data['HoleID'], y=data['AMPRD of ' + element], mode='markers', name=element, visible=False, marker=dict(color='blue')))
        fig.add_trace(go.Scatter(x=data['HoleID'], y=data['AMPRD of ' + element], mode='lines', name=element, visible=False, line=dict(dash='dash', color='blue')))
    
    # Set the first element visible by default
    fig.data[0].visible = True
    fig.data[1].visible = True
    
    # Create dropdown options for each element
    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)}, 
                     {'title': f"RDP vs DDH - {element}"}],
            'label': element,
            'method': 'update'
        })
        
        # Set only the corresponding scatter and line plots visible
        dropdown_buttons[i]['args'][0]['visible'][2*i] = True  # Scatter
        dropdown_buttons[i]['args'][0]['visible'][2*i + 1] = True  # Line
    
    # Add the dropdown to the layout
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 5}, 
        }],
        title=f"RDP vs DDH",
        width=900,
        height=600, 
    )
    
    fig.show()

# Define elements to plot
plot_data = [i for i in stat_data if 'Duplicate' not in i]

# Call the function with the list of elements
plot_rdp_vs_ddh(plot_data)


In [ ]:


# HTML and JS for the print button with print styles
html_code = """
    <style>
        @media print {
            @page {
                size: landscape; /* Set the page size to landscape */
            }
            body {
                transform: scale(0.9); /* Optional: Scale down the body if necessary */
                transform-origin: top left; /* Set the transform origin */
            }
        }
    </style>
    <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;" onclick="window.print()">Print this page</button>
"""

# Display the print button in the notebook, which will be included in the HTML output
display(HTML(html_code))